In [1]:
# mappper

def mapper(centoids, point):
    pass

In [2]:
a = [[1,2,3],[4,5,6]]
b = [[4,5,6],[1,2,3]]

In [3]:
a==b

False

In [4]:
a = [1,2,3]
print('key'+'\t',a)

key	 [1, 2, 3]


In [5]:
import pandas as pd

In [6]:
import numpy as np

In [17]:
file = open('shot_logs.csv')
c = 0
for line in file.readlines():
    line = line.strip()
    line = line.split(',')
    if line[0] == 'GAME_ID':
        continue
    shot_dist = line[12]
    close_def_dist = line[18]
    shot_clock = line[9]
    player_name = line[21]
    short_status = line[14]
    def_first_name = line[16][:-1]
    def_last_name = line[15][1:]
    print(line[14])
    print(def_first_name+"----"+def_last_name)
    if shot_clock == '':
        continue
    print(line)
    datapoint = np.array([shot_dist,close_def_dist,shot_clock])
    print('player_name: ',player_name)
    print('short status:',short_status)
    print(datapoint)

    c+=1
    if c==3:
        break
    file.close()

made
 Alan----Anderson
['21400899', '"MAR 04', ' 2015 - CHA @ BKN"', 'A', 'W', '24', '1', '1', '1:09', '10.8', '2', '1.9', '7.7', '2', 'made', '"Anderson', ' Alan"', '101187', '1.3', '1', '2', 'brian roberts', '203148']
player_name:  brian roberts
short status: made
['7.7' '1.3' '10.8']
missed
 Bojan----Bogdanovic
['21400899', '"MAR 04', ' 2015 - CHA @ BKN"', 'A', 'W', '24', '2', '1', '0:14', '3.4', '0', '0.8', '28.2', '3', 'missed', '"Bogdanovic', ' Bojan"', '202711', '6.1', '0', '0', 'brian roberts', '203148']
player_name:  brian roberts
short status: missed
['28.2' '6.1' '3.4']
missed
 Bojan----Bogdanovic
missed
 Markel----Brown
['21400899', '"MAR 04', ' 2015 - CHA @ BKN"', 'A', 'W', '24', '4', '2', '11:47', '10.3', '2', '1.9', '17.2', '2', 'missed', '"Brown', ' Markel"', '203900', '3.4', '0', '0', 'brian roberts', '203148']
player_name:  brian roberts
short status: missed
['17.2' '3.4' '10.3']


In [12]:
datapoint = datapoint.astype(np.float64)

In [9]:
datapoint

array([17.2,  3.4, 10.3])

In [10]:
centroid = np.array([28.2,6.1,3.4])
centroid.shape

(3,)

In [11]:
distance = np.zeros(4)

In [12]:
distance

array([0., 0., 0., 0.])

In [13]:
from numpy.linalg import norm

In [14]:
row = norm(datapoint - centroid, axis = 0 )

In [15]:
row

13.262729734108284

In [16]:
distance[1] = np.square(row)

In [17]:
distance

array([  0. , 175.9,   0. ,   0. ])

In [18]:
a = np.argmin(distance,axis=0)

In [19]:
a = list()

In [20]:
b = np.array([0,1])

In [21]:
a.append(b)

In [22]:
a

[array([0, 1])]

In [23]:
a.append(np.array([3,3]))

In [24]:
a

[array([0, 1]), array([3, 3])]

In [25]:
a = np.array(a)

In [26]:
a

array([[0, 1],
       [3, 3]])

In [27]:
a = np.sum(a,axis=0)

In [28]:
a

array([3, 4])

In [29]:
a = np.array([1,2])
b = np.array([3,4])

In [30]:
a + b

array([4, 6])

In [31]:
dict1 = {1: 1, 2: 9, 3: 4}

In [32]:
print(dict1)

{1: 1, 2: 9, 3: 4}


In [33]:
from operator import itemgetter

In [35]:
cv = dict()

In [55]:
#!usr/bin/python

import sys
import numpy as np
file = open('shot_logs.csv')
mapper_dict = dict()
c = 0
for line in file.readlines():

    line = line.strip()
    line = line.split(',')
    c+=1
    if c==1000:
        break

    # print(line)
    if line[0] == 'GAME_ID':
        continue
    shot_clock = line[9]
    player_name = line[21].lower()
    def_first_name = line[16][:-1]
    def_last_name = line[15][1:]
    def_name = def_first_name + " " + def_last_name
    shot_status = line[14]   # made / missed
    
    if shot_clock =='':   # skip first row, and, shot_clock is the only column with missing values
        continue

    if player_name not in mapper_dict:
        mapper_dict[player_name] = dict()

    if def_name not in mapper_dict[player_name]:
        mapper_dict[player_name][def_name] = list()

    if shot_status == 'made':
        mapper_dict[player_name][def_name].append(np.array([1,0]))
    else:
        mapper_dict[player_name][def_name].append(np.array([0,1]))

    
# now we have a dictionary of this kind:

# {brian roberts : {alan anderson : [[1,0],[1,0],[0,1],[0,1],[0,1]]
#                   bojan bogdanaovic : [[1,0],[0,1]] }}
# for all the players and their defenders with their short statuses

# combiner - let's aggregate all the sots made and missed so as to calculate the hit rate
# in reducer

for key, val in mapper_dict.items():
    # key is player_name
    # val is dictionary of defenders with individual shot statuses
    
    for k,v in val.items():
        # k is defender's name
        # v is a list of np arrays of short status
        v = np.array(v)
        v = np.sum(v,axis = 0)
        val[k] = v  # get sum of all those short statuses in one single array


# now or dictionary is ready for output to reducer
reducer_input = list()
for key, val in mapper_dict.items():

    for k,v in val.items():

        a = key + "\t" + k+"_"+str(v[0])+"_"+str(v[1])
        print(a)
        reducer_input.append(a)


# outputs like brian roberts         alan anderson_45_23
# in reducer make sure split is not based off the space character, but must be on the tab character

brian roberts	 Alan Anderson_1_0
brian roberts	 Bojan Bogdanovic_2_1
brian roberts	 Markel Brown_0_1
brian roberts	 Thaddeus Young_1_3
brian roberts	 Deron Williams_1_1
brian roberts	 Jarrett Jack_1_2
brian roberts	 Mason Plumlee_1_0
brian roberts	 Darius Morris_0_1
brian roberts	 Wayne Ellington_0_1
brian roberts	 Jeremy Lin_1_1
brian roberts	 Jordan Hill_1_0
brian roberts	 Willie Green_1_2
brian roberts	 Marcus Smart_0_2
brian roberts	 James Young_1_0
brian roberts	 Jonas Jerebko_0_2
brian roberts	 Jae Crowder_0_1
brian roberts	 Isaiah Thomas_3_4
brian roberts	 Aaron Brooks_2_3
brian roberts	 Kirk Hinrich_1_2
brian roberts	 Taj Gibson_2_0
brian roberts	 Joakim Noah_1_1
brian roberts	 Devin Harris_0_3
brian roberts	 Jose Juan Barea_0_3
brian roberts	 D.J. Augustin_2_3
brian roberts	 Andre Drummond_1_1
brian roberts	 John Lucas III_0_1
brian roberts	 Spencer Dinwiddie_0_1
brian roberts	 CJ Miles_1_0
brian roberts	 George Hill_2_1
brian roberts	 CJ Watson_3_5
brian roberts	 Shayne Whitt

In [37]:
v = [np.array([1,0]),np.array([0,1]),np.array([1,0])]

In [38]:
v

[array([1, 0]), array([0, 1]), array([1, 0])]

In [39]:
v = np.array(v)
v = np.sum(v,axis=0)
v

array([2, 1])

In [65]:
res = dict()
from operator import itemgetter
for line in reducer_input:

    line = line.strip()
    line = line.split('\t')

    player_name = line[0]

    gibberish = line[1].split('_')

    def_name = gibberish[0]
    made = int(gibberish[1])
    missed = int(gibberish[2])

    # print('player_name:',player_name)
    # print('def_name:',def_name)
    # print('made:',made)
    # print('missed:',missed)

    if player_name not in res:
        res[player_name] = dict()

    # print(res)
    

    res[player_name][def_name] = res[player_name].get(def_name,np.array([0,0])) + np.array([made,missed])

# now we have aggregate of all the mappers
# calculate hit rate now

for key, val in res.items():

    # key is player name
    # val is a dictionary with defender names and their made miss counts

    for k,v in val.items():
        # k is defender name
        md = v[0]
        mis = v[1]
        hit_rate = md/(md+mis)
        val[k] = hit_rate

# now the dictionary contains hit rate for each defender for each shot taken by a player

for key , val in res.items():

    sorted_defenders = sorted(val.items(), key = itemgetter(1))
    res[key] = sorted_defenders


# now for each player, just print the defender with lowest hit rate

for key, val in res.items():

    print(key, "  Worst Defender: -->", val[0][0])


brian roberts   Worst Defender: -->  Markel Brown
203148   Worst Defender: --> 240 ene
bismack biyombo   Worst Defender: -->  Jeremy Lin
al jefferson   Worst Defender: -->  Thaddeus Young
2744   Worst Defender: --> 240 ene
